# Data set

We generate our first data set using numpy random generator.

In [ ]:
import numpy as np

In [ ]:
n_samples = 60

np.random.seed(0)
X = 1*np.random.random((n_samples,2))
y = (X[:,1] > 0.5).astype(int)
linearly_separable = (X, y)

In [ ]:
data_xs, data_ys = linearly_separable

Data visualization tools using `plotly` are implemented in `activity_1_figure_utils`. We will just use them without explaining how they work. Feel free to ask questions about those during the workshop if you are interested.

In [ ]:
from activity_1_figure_utils import data_figure

In [ ]:
fig = data_figure(data_xs,data_ys)
fig.show()

# Data embedding quantum circuit

Let's use `qiskit` to construct our first data embedding circuit.

![data embedding circuit](notebook_ressources/Data_circuit.png "Data embedding circuit")

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector, Parameter

In [ ]:
def build_data_embedding_circuit():
    """Builds a plain 2D data embedding circuit

    Returns:
        QuantumCircuit: The data embedding parametrized quantum circuit
        ParameterVector or list[Parameter] : The parameters to be used to embed the data
    """

    data_params = ParameterVector('x', 2)
    
    ### Let's code here
    data_embedding_circuit = QuantumCircuit(1)
    
    data_embedding_circuit.ry(data_params[0],0)
    data_embedding_circuit.rz(data_params[1],0)
    ###

    return data_embedding_circuit, data_params

In [ ]:
data_embedding_circuit, data_params = build_data_embedding_circuit()
data_embedding_circuit.draw('mpl',scale = 2)

Note : All methods of this notebook are already implemented in `activity_1_circuits`, `activity_1_utils`.

In [ ]:
from activity_1_circuits import build_data_embedding_circuit

# Actually embedding the data

Let's create a method that inputs all the data points into the data embedding circuit. This should produce a `QuantumCircuit` for each data point.

In [ ]:
def embed_data(parametrized_circuit,data_params,data_xs):
    
    data_circuits = list()
    ### Let's code here
    for data_x in data_xs:
        data_x_dict = {p:v for (p,v) in zip(data_params,data_x)}
        data_circuit = parametrized_circuit.bind_parameters(data_x_dict)
        data_circuits.append(data_circuit)
    
    ###
    
    return data_circuits

In [ ]:
data_circuits = embed_data(data_embedding_circuit,data_params,data_xs)
data_circuits[0].draw('mpl',scale = 2)

Where does this data embedding puts the data point on the Bloch sphere?

In [ ]:
from activity_1_figure_utils import circuits_to_statevectors, bloch_sphere_statevector_figure

In [ ]:
statevectors = circuits_to_statevectors(data_circuits)
fig = bloch_sphere_statevector_figure(statevectors,data_ys)
fig.show()

# Rotation model quantum circuit

Let's build the quantum circuit of our first model. This circuit should take care of the data embedding as well as the rotation.

![rotation model circuit](notebook_ressources/Rotation_model_circuit.png "Rotation model circuit")

In [ ]:
def build_rotation_model_circuit():
    """Builds the rotation model quantum circuit. 
    First embeds the data.
    Then rotates the data.

    Returns:
        QuantumCircuit: The rotation model quantum circuit
        ParameterVector or list[Parameter] : The parameters to be used to embed the data
        ParameterVector or list[Parameter] : The model's parameters
    """

    data_params = ParameterVector('x', 2)
    rotation_params = ParameterVector('m', 2)

    model_circuit = QuantumCircuit(1)
    ### Let's code here
    model_circuit.ry(data_params[0],0)
    model_circuit.rz(data_params[1]+rotation_params[0],0)
    model_circuit.ry(rotation_params[1],0)
    ### Let's code here
    
    return model_circuit, data_params, rotation_params

In [ ]:
classifier_circuit, data_params, model_params = build_rotation_model_circuit()
classifier_circuit.draw('mpl',scale = 2)

# Build the layered model

Let's test our skills to build the layered model quantum circuit!

![layered model circuit](notebook_ressources/Layered_model_circuit.png "Layered model circuit")

$$f_i(x) = w_i x + \theta_i$$

In [ ]:
def build_layered_model_circuit(n_layers = 1):
    """Builds the layered model quantum circuit. 
    Takes care of the weighted data embedding and the rotations on many layers.

    Args:
        n_layers (int, optional): The number of layers. Defaults to 1.

    Returns:
        QuantumCircuit: The layered model quantum circuit
        ParameterVector or list[Parameter] : The parameters to be used to embed the data
        ParameterVector or list[Parameter] : The model's parameters (includes rotations and weights)
    """

    data_params = ParameterVector('x', 2)
    weights_params = ParameterVector('w', 2*n_layers)
    rotation_params = ParameterVector('m', 2*n_layers)

    model_params = list(rotation_params) + list(weights_params)
    
    model_circuit = QuantumCircuit(1)
    ### Let's code here
    model_circuit.ry(weights_params[0]*data_params[0],0)
    for l in range(0,n_layers-1):
        model_circuit.rz(weights_params[2*l+1] * data_params[1] + rotation_params[2*l+1],0)
        model_circuit.ry(weights_params[2*l+2] * data_params[0] + rotation_params[2*l+2],0)

    model_circuit.rz(weights_params[2*l+3] * data_params[1] + rotation_params[2*l+3],0) #2*n_layers-1
    model_circuit.ry(rotation_params[0],0)
    ###
    
    return model_circuit, data_params, model_params

In [ ]:
classifier_circuit, data_params, model_params = build_layered_model_circuit(n_layers = 3)
classifier_circuit.draw('mpl',scale = 2)

# Prepare all the circuits with parameter values and data points

We need to improve the `embed_data` method to input the model parameter values as well. We need the option to apply measurements or not for later uses.

In [ ]:
def prepare_all_circuits(model_circuit,data_params,model_params,data_xs,model_values,add_measurements=False):
    """Replace the model parameters of a parametrized QuantumCircuit with parameter values.
    Then replace the parameters of a parametrized QuantumCircuit with data.
    Produce a list of QuantumCircuit, one per data point.

    Args:
        model_circuit ([type]): The model circuit
        data_params ([type]): Parameters where to input the data
        model_params ([type]): Parameters where to input the model parameter values
        data_xs ([type]): Data points.
        model_values ([type]): Model parameter values
        add_measurements (bool, optional): Add a measurement at the end of the circuit. Defaults to False.

    Returns:
        List of QuantumCircuit: One QuantumCircuit per data point.
    """

    model_value_dict = {p:v for (p,v) in zip(model_params, model_values)}
    classifier_circuit = model_circuit.bind_parameters(model_value_dict)
    if add_measurements:
        classifier_circuit.measure_all()
        
    all_circuits = embed_data(classifier_circuit,data_params,data_xs)
    
    return all_circuits

In [ ]:
# classifier_circuit, data_params, model_params = build_rotation_model_circuit()
classifier_circuit, data_params, model_params = build_layered_model_circuit(n_layers = 3)
all_circuits = prepare_all_circuits(classifier_circuit,data_params,model_params,data_xs,[1,]*len(model_params),add_measurements=False)
all_circuits[0].draw('mpl',scale = 2)

# Running the circuits on simulators

We now want to run these quantum circuit on different simulator.

In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance

qasm_simulator = Aer.get_backend('qasm_simulator')
qasm_quantum_instance = QuantumInstance(qasm_simulator,shots=1000)

statevector_simulator = Aer.get_backend('statevector_simulator')
sv_quantum_instance = QuantumInstance(statevector_simulator)

In [ ]:
all_circuits = prepare_all_circuits(classifier_circuit,data_params,model_params,data_xs,[1,]*len(model_params),add_measurements=True)
all_results = qasm_quantum_instance.execute(all_circuits)
print(all_results.get_counts())

In [ ]:
all_circuits = prepare_all_circuits(classifier_circuit,data_params,model_params,data_xs,[1,]*len(model_params),add_measurements=False)
all_results = sv_quantum_instance.execute(all_circuits)
print(all_results.get_statevector(0))

We now need to convert the results (counts or statevectors) into expectation values of the Z operator.

In [ ]:
def all_results_to_expectation_values(all_results):
    """Convert results from running a list 1 qubit QuantumCircuit into Z expectation values.
    Select between statevector and counts method based on the backend used

    Args:
        all_results : Results from runnin all the circuit

    Returns:
        np.array: All the Z expectation values.
    """

    if all_results.backend_name == 'statevector_simulator':
        return all_statevectors_to_expectation_values(all_results)
    else:
        return all_counts_to_expectation_values(all_results.get_counts())


def all_counts_to_expectation_values(all_counts):
    """Convert a list of 1 qubit QuantumCircuit counts into Z expectation values.
    Results from the qasm_simulator or an actual backend.

    Args:
        all_counts (list of dict): The counts resulting of running all the QuantumCircuit. One per data point.

    Returns:
        np.array: All the Z expectation values.
    """

    n_data = len(all_counts)
    expectation_values = np.zeros((n_data,))
    eigenvalues = {'0': 1, '1': -1}
    for i, counts in enumerate(all_counts):
        ### Let's code here
        tmp1, tmp2 = 0, 0
        for key, value in counts.items():
            tmp1 += value * eigenvalues[key]
            tmp2 += value
        expectation_values[i] = tmp1/tmp2
        ###

    return expectation_values

def all_statevectors_to_expectation_values(all_results):
    """Convert the statevectors resulting of the simulation of a list of 1 qubit QuantumCircuit into Z expectation values.
    Results from the statevector_simulator.

    Args:
        all_counts (list of dict): The result of running all the QuantumCircuit.

    Returns:
        np.array: All the Z expectation values.
    """

    n_circuits = len(all_results.results)
    all_statevectors = np.zeros((n_circuits,2),dtype = complex)
    for i in range(n_circuits):
        all_statevectors[i,:] = all_results.get_statevector(i)

    pauli_z_eig = np.array([1.,-1.])
    expectation_values = np.real(np.einsum('ik,ik,k->i',all_statevectors,np.conjugate(all_statevectors),pauli_z_eig))

    return expectation_values

In [ ]:
expected_values = all_results_to_expectation_values(all_results)
print(expected_values)

# Evaluation of the cost function

The expectation values and the target values can be used to quantify the cost function.

In [ ]:
def eval_cost_fct_quadratic(expectation_values,target_values):
    """Convert expectation values into cost using a quadratic distance.

    Args:
        expectation_values (np.array): Values between -1 and 1.
        target_values (np.array): Values -1 or 1

    Returns:
        [np.array]: The computed cost of each data point.
    """
    
    product_zt = expectation_values*target_values
    all_costs = ((1 - product_zt)/2)**2
    return all_costs

In [ ]:
target_values = 1 - 2*data_ys
all_costs = eval_cost_fct_quadratic(expected_values,target_values)
print(all_costs)

# Train the model

We now use an optimizer to find the best model parameter values which minimize the total cost function. Hopefully this will allow us to have a good classifier.

In [ ]:
def train_classifier(optimizer,eval_cost_fct,quantum_instance,model_circuit,data_params,model_params,data_xs,data_ys,initial_point):
    """Train a classification model quantum circuit.

    Args:
        optimizer (Qiskit Optimizer): The optimizer used to minimize the cost function
        eval_cost_fct (function): Computes the cost of data points given expectation values and target values
        quantum_instance (Qiskit QuantumInstance): On which to run the QuantumCircuits.
        model_circuit (QuantumCircuit): The parametrized QuantumCircuit model.
        data_params ([type]): Parameters where to input the data
        model_params ([type]): Parameters where to input the model parameter values
        data_xs ([type]): Input data points
        data_ys ([type]): Class data points (0 or 1)
        initial_point ([type]): Initial set of parameters for the model

    Returns:
        model_values [list]: Optimal parameter values found by the optimizer
        loss [float]: Final cost value
        nfev [int]: Number of iteration done by the optimizer
    """

    target_values = 1 - 2*data_ys

    add_measurements = quantum_instance.backend_name != 'statevector_simulator'
    
    def cost_function(model_values):

        all_circuits = prepare_all_circuits(model_circuit,data_params,model_params,data_xs,model_values,add_measurements)
        all_results = quantum_instance.execute(all_circuits)
        expectation_values = all_results_to_expectation_values(all_results)
        all_costs = eval_cost_fct(expectation_values,target_values)
        return np.sum(all_costs)/len(all_costs)
    
    model_values, loss, nfev = optimizer.optimize(len(model_params), cost_function, initial_point=initial_point)

    return model_values, loss, nfev

In [ ]:
from qiskit.algorithms.optimizers import SPSA, COBYLA, SLSQP
from activity_1_utils import spsa_optimizer_callback
from activity_1_figure_utils import history_figure
from activity_1_circuits import build_rotation_model_circuit, build_linear_model_circuit, build_layered_model_circuit
import time

We can choose the model we want to use.

In [ ]:
# model = 'rotation'
model = 'linear'
# model = 'layered'
if model == 'rotation':
    classifier_circuit, data_params, model_params = build_rotation_model_circuit()
    initial_point = [0,0]
elif model == 'linear':
    classifier_circuit, data_params, model_params = build_linear_model_circuit()
    initial_point = [0,0,1,1]
elif model == 'layered':
    n_layers = 4
    classifier_circuit, data_params, model_params = build_layered_model_circuit(n_layers)
    initial_point = [0,0] * n_layers + [1,1] * n_layers

An now we train it!

In [ ]:
t0 = time.time()

train_history = []
optimizer = SPSA(maxiter=50, callback=lambda n, p, v, ss, sa: spsa_optimizer_callback(n, p, v, ss, sa, train_history))

model_values, loss, nfev = train_classifier(
    optimizer,eval_cost_fct_quadratic,sv_quantum_instance,
    classifier_circuit,data_params,model_params,data_xs,data_ys,initial_point
    )

fig = history_figure(train_history)
fig.show()
print(model_values)
print(f'{time.time() - t0:.2f} sec')

In [ ]:
all_circuits = prepare_all_circuits(classifier_circuit,data_params,model_params,data_xs,model_values,add_measurements=False)
statevectors = circuits_to_statevectors(all_circuits)
fig = bloch_sphere_statevector_figure(statevectors,data_ys)
fig.show()

# Using the model to classify

Now that we have good model parameter values, let's use them to classify.

In [ ]:
def all_results_to_classifications(all_results):
    """Convert result into class

    Args:
        all_results ([type]): Results from running QuantumCircuits

    Returns:
        np.array: Prediction class (0 or 1)
    """
    
    expectation_values = all_results_to_expectation_values(all_results)
    classifications = np.choose(expectation_values>0,[1,0])

    return classifications


def classify(quantum_instance,model_circuit,model_params,model_values,data_params,data_xs):
    """Classify data point given a model, model values and a backend.

    Args:
        quantum_instance (Qiskit QuantumInstance): On which to run the QuantumCircuits.
        model_circuit (QuantumCircuit): The parametrized QuantumCircuit model.
        model_params ([type]): Parameters where to input the model parameter values
        model_values ([type]): Parameter values to be used into the model
        data_params ([type]): Parameters where to input the data
        data_xs ([type]): Input data points

    Returns:
        np.array: Prediction class (0 or 1)
    """

    add_measurements = quantum_instance.backend_name != 'statevector_simulator'

    all_circuits = prepare_all_circuits(model_circuit,data_params,model_params,data_xs,model_values,add_measurements)
    all_results = quantum_instance.execute(all_circuits)
    classifications = all_results_to_classifications(all_results)

    return classifications

In [ ]:
from activity_1_figure_utils import classification_figure

In [ ]:
predictions_ys = classify(sv_quantum_instance,classifier_circuit,model_params,model_values,data_params,data_xs)
fig = classification_figure(data_xs,data_ys,predictions_ys)
fig.show()

# Running on an actual quantum computer

Let's give an example on how to run QuantumCircuit on actual quantum computer!

In [ ]:
from qiskit import IBMQ
from qiskit.visualization import plot_error_map

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education',group='qscitech-quantum',project='qc-bc-workshop')

In [ ]:
ibmq_jakarta = provider.get_backend('ibmq_jakarta')
plot_error_map(ibmq_jakarta)

In [ ]:
ibmq_quantum_instance = QuantumInstance(ibmq_jakarta,shots=8192,initial_layout=[4,])

In [ ]:
predictions_ys = classify(ibmq_quantum_instance,classifier_circuit,model_params,model_values,data_params,data_xs)
fig = classification_figure(data_xs,data_ys,predictions_ys)
fig.show()

# Notes
Created by Maxime Dion <maxime.dion@usherbrooke.ca>